In [49]:
import pandas as pd
import pickle
import hashlib

In [79]:
with open(r"C:\Development\ultimateNakMuay\notebooks\pickle_objects\fighter_ufc_payload_async.pkl", "rb") as f:
    results = pickle.load(f)

In [80]:
def cleanResults(result):
    event_name = list(result)[0]
    event_df = result[event_name][0]
    event_df.columns = [_[1] for _ in event_df.columns]
    sub_dfs = []
    start_index = 0

    for i, row in event_df.iterrows():
        if ("Preliminary card" in row["Weight class"]) or (("Preliminary Card" in row["Weight class"])):
            sub_df = event_df[start_index:i]
            sub_dfs.append(sub_df)
            start_index = i + 1

    sub_df = event_df[start_index:]
    sub_dfs.append(sub_df)
    
    for i, sdf in enumerate(sub_dfs):
        if i == 0:
            sdf.loc[:,"event_type"] = "main event"
        else:
            sdf.loc[:,"event_type"] = "preliminary event"
    
    df = pd.concat(sub_dfs)

    cols2rename = {
        x: x.lower().replace(' ', '_') for x in df.columns
    }

    cols2rename = {
        **cols2rename,
        **{
            "Unnamed: 1_level_1": "winner",
            "Unnamed: 3_level_1": "loser",
        },
    }

    df.rename(columns=cols2rename, inplace=True)

    df.drop(columns=["unnamed:_2_level_1"], axis=1, inplace=True)

    df.loc[:, "event_name"] = event_name

    df.loc[:, "event_id"] = df["event_name"].apply(lambda x: hashlib.sha256(x.encode()).hexdigest())

    return df

In [81]:
results = list(map(cleanResults, results))

C:\Users\paulm\AppData\Local\Temp\ipykernel_14196\1973832475.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf.loc[:,"event_type"] = "main event"
C:\Users\paulm\AppData\Local\Temp\ipykernel_14196\1973832475.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf.loc[:,"event_type"] = "preliminary event"
C:\Users\paulm\AppData\Local\Temp\ipykernel_14196\1973832475.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [83]:
final_df = pd.concat(results, ignore_index=True)

In [84]:
final_df

,weight_class,winner,loser,method,round,time,notes,event_type,event_name,event_id
0,Light Heavyweight,Anthony Smith,Maurício Rua,KO (elbow and punches),1,1:29,NaN,main event,UFC Fight Night: Shogun vs. Smith,713c24786b3eb2030436621a470bfc8a1dd99c6ff1e5f0...
1,Light Heavyweight,Corey Anderson,Glover Teixeira,"Decision (unanimous) (30–27, 30–27, 30–27)",3,5:00,NaN,main event,UFC Fight Night: Shogun vs. Smith,713c24786b3eb2030436621a470bfc8a1dd99c6ff1e5f0...
2,Middleweight,Abu Azaitar,Vitor Miranda,"Decision (unanimous) (30–27, 30–27, 29–28)",3,5:00,NaN,main event,UFC Fight Night: Shogun vs. Smith,713c24786b3eb2030436621a470bfc8a1dd99c6ff1e5f0...
3,Heavyweight,Marcin Tybura,Stefan Struve,"Decision (unanimous) (30–27, 30–27, 29–28)",3,5:00,NaN,main event,UFC Fight Night: Shogun vs. Smith,713c24786b3eb2030436621a470bfc8a1dd99c6ff1e5f0...
4,Welterweight,Danny Roberts,David Zawada,"Decision (split) (27–30, 29–28, 29–28)",3,5:00,NaN,main event,UFC Fight Night: Shogun vs. Smith,713c24786b3eb2030436621a470bfc8a1dd99c6ff1e5f0...
...,...,...,...,...,...,...,...,...,...,...
108,Heavyweight,Timothy Johnson,Daniel Omielańczuk,"Decision (split) (28–29, 30–27, 29–28)",3,5:00,NaN,preliminary event,UFC Fight Night: Manuwa vs. Anderson,31dc582f14b951c93df89f63cfa062b02eeade9295ba24...
109,Welterweight,Leon Edwards,Vicente Luque,"Decision (unanimous) (29–28, 29–28, 29–28)",3,5:00,NaN,preliminary event,UFC Fight Night: Manuwa vs. Anderson,31dc582f14b951c93df89f63cfa062b02eeade9295ba24...
110,Lightweight,Marc Diakiese,Teemu Packalén,KO (punch),1,0:30,NaN,preliminary event,UFC Fight Night: Manuwa vs. Anderson,31dc582f14b951c93df89f63cfa062b02eeade9295ba24...
111,Middleweight,Brad Scott,Scott Askham,"Decision (split) (28–29, 29–28, 29–28)",3,5:00,NaN,preliminary event,UFC Fight Night: Manuwa vs. Anderson,31dc582f14b951c93df89f63cfa062b02eeade9295ba24...
